In [ ]:
from azureml.core.environment import Environment
from azureml.core.workspace import Workspace
from azureml.core.conda_dependencies import CondaDependencies

ws = Workspace.from_config()

# to install required packages
env = Environment('diabetes-encryption-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults', 'azure-storage-blob', 'encrypted-inference==0.9'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
!pip install encrypted-inference==0.9

In [ ]:
%%writefile score_enc.py
import json
import os
import pickle
import joblib
import numpy as np
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, PublicAccess
from encrypted.inference.eiserver import EIServer

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes-sklearn-model.pkl')
    model = joblib.load(model_path)

    global server
    print(model.coef_)
    print(model.intercept_)

    coeffs = np.asarray(model.coef_)
    coeffs = coeffs.reshape(1,-1)
    print(coeffs.shape)
    print(coeffs)

    intercept = np.asarray(model.intercept_)
    intercept = intercept.reshape(-1)
    print(intercept.shape)
    print(intercept)

    #server = EIServer(model.coef_, model.intercept_, verbose=True)
    server = EIServer(coeffs, intercept, verbose=True)

def run(raw_data):

    json_properties = json.loads(raw_data)

    key_id = json_properties['key_id']
    conn_str = json_properties['conn_str']
    container = json_properties['container']
    data = json_properties['data']

    # download the Galois keys from blob storage
    #TODO optimize by caching the keys locally  
    blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)
    blob_client = blob_service_client.get_blob_client(container=container, blob=key_id)
    public_keys = blob_client.download_blob().readall()
    
    result = {}
    # make prediction
    result = server.predict(data, public_keys)

    # you can return any data type as long as it is JSON-serializable
    return result

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "diabetes",  "method" : "sklearn"}, 
                                               description='Encrypted Predict diabetes with sklearn + SEAL')

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'diabetes_sklearn')

myenv = Environment.get(workspace=ws, name="diabetes-encryption-env")
inference_config = InferenceConfig(entry_script="score_enc.py", environment=myenv)

service_name = 'diabetes-enc-svc-' + str(uuid.uuid4())[:4]
print('Deploying service = ', service_name)
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [ ]:
print(service.scoring_uri)

In [ ]:
import os
import pandas as pd
import numpy as np

data = pd.read_csv("diabetes-test.csv")
print(data.shape)
feature_names = data.iloc[:,0:8].columns
target_name = data.iloc[:1,8:].columns
print(feature_names)
print(target_name)

data_features = data[feature_names]
X_test = data_features.to_numpy(dtype=np.uint8)
data_target = data[target_name]
y_test = data_target.to_numpy(dtype=np.uint8).reshape(-1)

print(X_test)
print(y_test)


In [ ]:
from encrypted.inference.eiclient import EILinearRegressionClient

# Create a new Encrypted inference client and a new secret key.
edp = EILinearRegressionClient(verbose=True)

public_keys_blob, public_keys_data = edp.get_public_keys()


In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
import os

ws = Workspace.from_config()

datastore = ws.get_default_datastore()
container_name=datastore.container_name

# Create a local file and write the keys to it
public_keys = open(public_keys_blob, "wb")
public_keys.write(public_keys_data)
public_keys.close()

# Upload the file to blob store
datastore.upload_files([public_keys_blob])

# Delete the local file
os.remove(public_keys_blob)

In [ ]:
#choose any one sample from the test data 
random_index = np.random.randint(0, len(X_test)-1)
print('Random index = ', random_index)
print(X_test[random_index])

single_sample = X_test[random_index].reshape(1,-1)
print(single_sample)
print(single_sample.tolist())

#encrypt the data
raw_data = edp.encrypt(X_test[random_index])
#raw_data = edp.encrypt(single_sample.tolist())
#print(raw_data)


test_decrypt = edp.decrypt(raw_data)
print(test_decrypt)


In [ ]:
import json
from azureml.core import Webservice

service = Webservice(ws, service_name)

#pass the connection string for blob storage to give the server access to the uploaded public keys 
conn_str_template = 'DefaultEndpointsProtocol={};AccountName={};AccountKey={};EndpointSuffix=core.windows.net'
conn_str = conn_str_template.format(datastore.protocol, datastore.account_name, datastore.account_key)

#build the json 
data = json.dumps({"data": raw_data, "key_id" : public_keys_blob, "conn_str" : conn_str, "container" : container_name })
data = bytes(data, encoding='ASCII')

print ('Making an encrypted inference web service call ')
eresult = service.run(input_data=data)

print ('Received encrypted inference results')
print(eresult)

In [ ]:
import numpy as np 

results = edp.decrypt(eresult)

print ('Decrypted the results ', results)

#Apply argmax to identify the prediction result
#prediction = np.argmax(results)
if results >= 0.5:
    prediction = 1
else:
    prediction = 0

print ( ' Prediction : ', prediction)
print ( ' Actual Label : ', y_test[random_index])

In [ ]:
service.delete()

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/tutorials/img-classification-part2-deploy.png)